# CHPASE GATE!

New entry point. Should be easier to keep track of parameters and namespace issues using this new method

## Preamble

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from Circuit_Objs.qchard_coupobj import *
from Circuit_Objs.qchard_evolgates import *
from workflow_funcs import *

## Workflows

### Abstracted Workflow

#### Fluxonium-IdealGridium

In [ ]:
# Turn this into a seperate dedicated function
def scale_adjust(system:CoupledObjects, system_cfg:SystemConfig):
    # Takes in two qubits and returns a parameter scaled version of qubitA to fit the transition frequencies
    E1 = system.level(system_cfg.coupled_resonant_transitions[0], interaction='off')
    E2 = system.level(system_cfg.coupled_resonant_transitions[1], interaction='off')
    splitting = E2-E1

fluxonium = Fluxonium(**heavy_fluxonium_params, **std_fluxonium_sim_params)
gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
grid_driven_tran = (gridium.level(5) - gridium.level(3))
fluxonium_init_trans = fluxonium.level(1) - fluxonium.level(0)
print(fluxonium_init_trans)
scaling = grid_driven_tran/fluxonium_init_trans
new_fluxonium_params = dict()
for key, value in heavy_fluxonium_params.items():
    new_fluxonium_params[key] = value*scaling
new_fluxonium = Fluxonium(**new_fluxonium_params, **std_fluxonium_sim_params)
print(new_fluxonium.level(1) - new_fluxonium.level(0))

In [ ]:
# fluxonium = Fluxonium(**fluxonium_params, **std_fluxonium_sim_params)
fluxonium = new_fluxonium
gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
gridium.nlev = 8
fluxonium.nlev = 5
# transmon01_freq = gridium.transition_energies()[3+1]
# transmon = transmon_creation_from_01(linear_freq=transmon01_freq)

pulse_path = 'yamls/pulses/fluxonium_idealgridium.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_idealgridium.yaml'
fig = solve_coupled_qubits(fluxonium, gridium, pulse_path=pulse_path, syscfg_path=syscfg_path, n_shown_states=3)
fig.show()

It seems as if the rate limiting step for simulations that have high ending fidelity is actually simulating the propegator for all combinations of levels (typically 10x10). This does not seem to meaningfully change the graphs, so for trial simulations, truncated nlevs seems to be appropriate. For very complex/large qubit simulations, best practice is to first simulate the eigenvectors/eigenvalues before then saving the object. This saves sucessive solve times for later.



### Modular Workflow

When you want to get more in the weeds. Basically the contents of solve_coupled_qubits

#### Transmon-IdealGridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')

import yaml
from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/std.yaml'
syscfg_path = 'yamls/syscfgs/transmon_idealgridium.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
transmon01_freq = gridium.transition_energies()[4]
transmon = transmon_creation_from_01(linear_freq=transmon01_freq)

gridium.nlev = 8
transmon.nlev = 4
qubitA = transmon
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
xopt, infidelity = minimize_infidelity(system, pulse_cfg, system_cfg)

#### SCQTransmon-IdealGridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')

import yaml
from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

pulse_path = 'yamls/pulses/std.yaml'
syscfg_path = 'yamls/syscfgs/std.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
transmon = SCQTransmon(0.3,15,0,5,20)

grid_driven_tran = (gridium.level(5) - gridium.level(3))
fluxonium_init_trans = transmon.level(1) - transmon.level(0)
print(fluxonium_init_trans)
print(grid_driven_tran)
scaling = grid_driven_tran/fluxonium_init_trans
print(scaling)
transmon.E_C = transmon.E_C*scaling
transmon.E_J = transmon.E_J*scaling
print(transmon.level(1) - transmon.level(0))

gridium.nlev = 8
qubitA = transmon
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
xopt, infidelity = minimize_infidelity(system, pulse_cfg, system_cfg)

#### Fluxonium-IdealGridium -- Soft

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

In [ ]:
fluxonium = Fluxonium(**heavy_fluxonium_params, **std_fluxonium_sim_params)
gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
grid_driven_tran = (gridium.level(5) - gridium.level(3))
fluxonium_init_trans = fluxonium.level(1) - fluxonium.level(0)
print(fluxonium_init_trans)
scaling = grid_driven_tran/fluxonium_init_trans
new_fluxonium_params = dict()
for key, value in heavy_fluxonium_params.items():
    new_fluxonium_params[key] = value*scaling
new_fluxonium = Fluxonium(**new_fluxonium_params, **std_fluxonium_sim_params)
print(new_fluxonium.level(1) - new_fluxonium.level(0))

In [ ]:
fluxonium = new_fluxonium
gridium.nlev = 8
fluxonium.nlev = 5

pulse_path = 'yamls/pulses/fluxonium_idealgridium_soft.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_idealgridium_soft.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

qubitA = fluxonium
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
xopt, cphase_pi_error_value = converge_on_pi(system, pulse_cfg, system_cfg)

In [ ]:
xopt, infidelity = minimize_infidelity(system, pulse_cfg, system_cfg)

#### Fluxonium-IdealGridium -- Hard 

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

In [ ]:
fluxonium = Fluxonium(**heavy_fluxonium_params, **std_fluxonium_sim_params)
gridium = IdealGridium(**hard_IdealGridium_params, **std_IdealGridium_sim_params)
gridium.nlev = 8
gridium.nlev_lc = 1200

In [ ]:
def rescaling(system:CoupledObjects, system_cfg:SystemConfig):
    _, unchanged_transition = (system.level(system_cfg.transitions_to_drive[0], interaction='off')
                          - system.level(system_cfg.transitions_to_drive[1], interaction='off'))
    pre_changed_transition, _ = (system.level(system_cfg.transitions_to_drive[0], interaction = 'off')
                           - system.level(system_cfg.detuned_transitions[0], interaction = 'off'))
    scaling = unchanged_transition/pre_changed_transition
    print("Qubit A's original transition is {} GHz".format(pre_changed_transition))
    print("Qubit B's transition is {} GHz".format(unchanged_transition))
    

grid_driven_tran = (gridium.level(5) - gridium.level(3))
fluxonium_init_trans = fluxonium.level(1) - fluxonium.level(0)
print(fluxonium_init_trans)
print(grid_driven_tran)
scaling = grid_driven_tran/fluxonium_init_trans
new_fluxonium_params = dict()
for key, value in heavy_fluxonium_params.items():
    new_fluxonium_params[key] = value*scaling
new_fluxonium = Fluxonium(**new_fluxonium_params, **std_fluxonium_sim_params)
print(new_fluxonium.level(1) - new_fluxonium.level(0))
fluxonium = new_fluxonium


grid_driven_tran = (gridium.level(5) - gridium.level(3))
fluxonium_init_trans = fluxonium.level(1) - fluxonium.level(0)
print(fluxonium_init_trans)
print(grid_driven_tran)
scaling = grid_driven_tran/fluxonium_init_trans
new_fluxonium_params = dict()
for key, value in heavy_fluxonium_params.items():
    new_fluxonium_params[key] = value*scaling
new_fluxonium = Fluxonium(**new_fluxonium_params, **std_fluxonium_sim_params)
print(new_fluxonium.level(1) - new_fluxonium.level(0))
fluxonium = new_fluxonium

In [ ]:
pulse_path = 'yamls/pulses/fluxonium_idealgridium_hard.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_idealgridium_hard.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

qubitA = fluxonium
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
xopt, cphase_pi_error_value = converge_on_pi(system, pulse_cfg, system_cfg)

#### Shallow/Light Fluxonium-Soft Gridium

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
import yaml

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_fluxonium import *
from Circuit_Objs.qchard_transmon import *
from workflow_funcs import *

In [ ]:
dictionary = light_fluxonium_params
fluxonium = Fluxonium(**dictionary, **std_fluxonium_sim_params)
gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
gridium.nlev = 8

In [ ]:
grid_driven_tran = (gridium.level(5) - gridium.level(3))
fluxonium_init_trans = fluxonium.level(1) - fluxonium.level(0)
print(fluxonium_init_trans)
print(grid_driven_tran)
scaling = grid_driven_tran/fluxonium_init_trans
print(scaling)
new_fluxonium_params = dict()
for key, value in dictionary.items():
    new_fluxonium_params[key] = value*scaling
new_fluxonium = Fluxonium(**new_fluxonium_params, **std_fluxonium_sim_params)
print(new_fluxonium.level(1) - new_fluxonium.level(0))
fluxonium = new_fluxonium

In [ ]:
pulse_path = 'yamls/pulses/fluxonium_light_idealgridium_soft.yaml'
syscfg_path = 'yamls/syscfgs/fluxonium_light_idealgridium_soft.yaml'

with open(pulse_path, 'r') as f:
    data = yaml.safe_load(f)
    pulse_cfg = PulseConfig(**data)

with open(syscfg_path, 'r') as f:
    data = yaml.safe_load(f)
    system_cfg = SystemConfig(**data)
    del data

qubitA = fluxonium
qubitB = gridium
n_shown_states = 3

qubitA, qubitB = load_qubits(qubitA, qubitB)
system = couple_qubits(qubitA, qubitB, system_cfg, pulse_cfg, mute=False)
t_points, U_t, phase_accum, fidelity = solve(system, pulse_cfg, system_cfg, solve_method='propagator', mute=False)
fig = visualize_state_propagation(system, system_cfg, t_points, U_t, phase_accum, fidelity, n_shown_states=n_shown_states)

In [ ]:
xopt, cphase_pi_error_value = converge_on_pi(system, pulse_cfg, system_cfg)

In [ ]:
xopt, infidelity = minimize_infidelity(system, pulse_cfg, system_cfg)

## Saving Diagonalized Objects

For speed in later use

In [ ]:
# gridium = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
# gridium.nlev = 10
gridium.levels(eigvecs=True)
gridium.save_obj('/Users/thomasersevim/QNL/2q_gridium/etc/qubits/')

# fluxonium = Fluxonium(**fluxonium_params, **std_fluxonium_sim_params)
# fluxonium.nlev = 10
fluxonium.levels(eigvecs=True)
fluxonium.save_obj('/Users/thomasersevim/QNL/2q_gridium/etc/qubits/')

## Testing

In [ ]:
transmon = SCQTransmon(0.3, 15, 0, 6, 8)
transmon.nlev_lc = 1000
transmon.nlev = 20
print(transmon.transition_energies())